In [5]:
import librosa
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import json

In [23]:
initial_files = pd.read_csv('soundfile_names.csv')
initial_files

,filename,label
0,blues.00000.wav,blues
1,blues.00001.wav,blues
2,blues.00002.wav,blues
3,blues.00003.wav,blues
4,blues.00004.wav,blues
...,...,...
995,rock.00095.wav,rock
996,rock.00096.wav,rock
997,rock.00097.wav,rock
998,rock.00098.wav,rock


In [ ]:
def extract_features_per_segment(file_path, n_segments=10):
    try:
        y, sr = librosa.load(file_path)
    except:
        print(file_path, "has an error and is unable to be read")
        return {}
    
    segment_length = len(y) // n_segments  # Number of samples per segment
    segment_duration = segment_length / sr  # Duration of each segment in seconds

    # List to store features for each segment
    all_segments_features = []
    
    # Process each segment
    for i in range(n_segments):
        start = i * segment_length
        end = (i + 1) * segment_length if i < n_segments - 1 else len(y)
        y_segment = y[start:end]
        
        # Compute features for the segment
        features = {
            "chroma_stft": librosa.feature.chroma_stft(y=y_segment, sr=sr),
            "chroma_cqt": librosa.feature.chroma_cqt(y=y_segment, sr=sr),
            "chroma_cens": librosa.feature.chroma_cens(y=y_segment, sr=sr),
            "tonnetz": librosa.feature.tonnetz(y=y_segment, sr=sr),
            "rms": librosa.feature.rms(y=y_segment),
            "spectral_centroid": librosa.feature.spectral_centroid(y=y_segment, sr=sr),
            "spectral_bandwidth": librosa.feature.spectral_bandwidth(y=y_segment, sr=sr),
            "spectral_rolloff": librosa.feature.spectral_rolloff(y=y_segment, sr=sr),
            "zero_crossing_rate": librosa.feature.zero_crossing_rate(y=y_segment),
            "harmony": librosa.effects.harmonic(y_segment),
            "percussive": librosa.effects.percussive(y_segment),
        }
        
        # 20 MFCCs
        mfccs = librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=20)
        for j in range(20):
            features[f"mfcc_{j+1}"] = mfccs[j, :]

        # Calculate statistics for each feature and add length
        segment_features = {"filename": file_path, "segment": i, "length": segment_duration}  # Include filename and segment
        for key, values in features.items():
            segment_features[f"{key}_mean"] = np.mean(values)
            segment_features[f"{key}_std"] = np.std(values)
            segment_features[f"{key}_skew"] = skew(values, axis=None)
            segment_features[f"{key}_kurtosis"] = kurtosis(values, axis=None)
            segment_features[f"{key}_median"] = np.median(values)
            segment_features[f"{key}_min"] = np.min(values)
            #segment_features[f"{key}_max"] = np.max(values)

        segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
            #segment_features[f"{key}_vector"] = values  # Store the full vector

        all_segments_features.append(segment_features)

    return all_segments_features

In [6]:
file_paths = list("Data/genres_original/" + initial_files['label'] + "/" + initial_files['filename'])
file_paths

['Data/genres_original/blues/blues.00000.wav',
 'Data/genres_original/blues/blues.00001.wav',
 'Data/genres_original/blues/blues.00002.wav',
 'Data/genres_original/blues/blues.00003.wav',
 'Data/genres_original/blues/blues.00004.wav',
 'Data/genres_original/blues/blues.00005.wav',
 'Data/genres_original/blues/blues.00006.wav',
 'Data/genres_original/blues/blues.00007.wav',
 'Data/genres_original/blues/blues.00008.wav',
 'Data/genres_original/blues/blues.00009.wav',
 'Data/genres_original/blues/blues.00010.wav',
 'Data/genres_original/blues/blues.00011.wav',
 'Data/genres_original/blues/blues.00012.wav',
 'Data/genres_original/blues/blues.00013.wav',
 'Data/genres_original/blues/blues.00014.wav',
 'Data/genres_original/blues/blues.00015.wav',
 'Data/genres_original/blues/blues.00016.wav',
 'Data/genres_original/blues/blues.00017.wav',
 'Data/genres_original/blues/blues.00018.wav',
 'Data/genres_original/blues/blues.00019.wav',
 'Data/genres_original/blues/blues.00020.wav',
 'Data/genres

In [25]:
for i in initial_files['label'].unique():
    genre_labels = initial_files[initial_files['label'] == i]
    file_paths = list("Data/genres_original/" + genre_labels['label'] + "/" + genre_labels['filename'])
   
    files_features = [extract_features_per_segment(fp) for fp in file_paths]
    df = pd.DataFrame([item for sublist in files_features for item in sublist])
    df.to_csv(i + "_3secfeatures.csv", index = False)

    #all_files_features = [extract_features_per_segment(fp) for fp in file_paths]


/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0]

Data/genres_original/jazz/jazz.00054.wav has an error and is unable to be read


/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_27663/1565882270.py:49: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0]

# For new songs

In [1]:
import os

new_music_path = os.path.join(os.getcwd(), "Data", "music_extracted")
files = [i for i in os.walk(new_music_path)]

In [2]:
file_paths = []

first = True
for dir in files:
    if not first:
        songs = dir[2]
        for s in songs:
            file_paths.append(os.path.join(dir[0], s))
            
    else:
        first = False

In [6]:
def extract_features_per_segment(file_path):
    try:
        y, sr = librosa.load(file_path)
    except:
        print(file_path, "has an error and is unable to be read")
        return {}

    segment_duration = 3
    segment_length = segment_duration * sr
    n_segments = len(y) // segment_length - 1

    # List to store features for each segment
    all_segments_features = []
    
    # Process each segment
    for i in range(n_segments):
        start = i * segment_length
        end = (i + 1) * segment_length if i < n_segments - 1 else len(y)
        y_segment = y[start:end]
        
        # Compute features for the segment
        features = {
            "chroma_stft": librosa.feature.chroma_stft(y=y_segment, sr=sr),
            "chroma_cqt": librosa.feature.chroma_cqt(y=y_segment, sr=sr),
            "chroma_cens": librosa.feature.chroma_cens(y=y_segment, sr=sr),
            "tonnetz": librosa.feature.tonnetz(y=y_segment, sr=sr),
            "rms": librosa.feature.rms(y=y_segment),
            "spectral_centroid": librosa.feature.spectral_centroid(y=y_segment, sr=sr),
            "spectral_bandwidth": librosa.feature.spectral_bandwidth(y=y_segment, sr=sr),
            "spectral_rolloff": librosa.feature.spectral_rolloff(y=y_segment, sr=sr),
            "zero_crossing_rate": librosa.feature.zero_crossing_rate(y=y_segment),
            "harmony": librosa.effects.harmonic(y_segment),
            "percussive": librosa.effects.percussive(y_segment),
        }
        
        # 20 MFCCs
        mfccs = librosa.feature.mfcc(y=y_segment, sr=sr, n_mfcc=20)
        for j in range(20):
            features[f"mfcc_{j+1}"] = mfccs[j, :]

        # Calculate statistics for each feature and add length
        segment_features = {"filename": file_path, "segment": i, "length": segment_duration}  # Include filename and segment
        for key, values in features.items():
            segment_features[f"{key}_mean"] = np.mean(values)
            segment_features[f"{key}_std"] = np.std(values)
            segment_features[f"{key}_skew"] = skew(values, axis=None)
            segment_features[f"{key}_kurtosis"] = kurtosis(values, axis=None)
            segment_features[f"{key}_median"] = np.median(values)
            segment_features[f"{key}_min"] = np.min(values)
        segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
            #segment_features[f"{key}_vector"] = values  # Store the full vector

        all_segments_features.append(segment_features)

    return all_segments_features

In [7]:
files_features = [extract_features_per_segment(fp) for fp in file_paths]
df = pd.DataFrame([item for sublist in files_features for item in sublist])

/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:50: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/Users/carson/anaconda3/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  segment_features[f"{key}_skew"] = skew(values, axis=None)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:47: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This 

/Users/carson/aml-music/Data/music_extracted/classical/.DS_Store has an error and is unable to be read


/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:50: FutureWarning: librosa.beat.tempo
	This function was moved to 'librosa.feature.rhythm.tempo' in librosa version 0.10.0.
	This alias will be removed in librosa version 1.0.
  segment_features['tempo'] = float(librosa.beat.tempo(y=y_segment, sr=sr)[0])
/Users/carson/anaconda3/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  segment_features[f"{key}_skew"] = skew(values, axis=None)
/var/folders/q3/hm2wt3010319t7l0hz8w8s4h0000gn/T/ipykernel_32325/3752193276.py:47: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This 

In [14]:
def genre(x):
    return x.split('/')[6]

df['genre'] = df['filename'].apply(genre)

In [15]:
df

,filename,segment,length,chroma_stft_mean,chroma_stft_std,chroma_stft_skew,chroma_stft_kurtosis,chroma_stft_median,chroma_stft_min,chroma_cqt_mean,...,mfcc_19_median,mfcc_19_min,mfcc_20_mean,mfcc_20_std,mfcc_20_skew,mfcc_20_kurtosis,mfcc_20_median,mfcc_20_min,tempo,genre
0,/Users/carson/aml-music/Data/music_extracted/p...,0,3,0.401813,0.327726,0.183252,-1.106295,0.411137,0.000000,0.452390,...,-2.082096,-16.621260,0.774741,3.078783,0.017335,1.191523,0.000000,-10.270206,129.199219,pop
1,/Users/carson/aml-music/Data/music_extracted/p...,1,3,0.544555,0.251707,0.269058,-0.851386,0.515716,0.027985,0.600131,...,-5.715023,-19.307529,2.139871,4.793799,0.435874,-0.046180,2.019626,-7.701082,107.666016,pop
2,/Users/carson/aml-music/Data/music_extracted/p...,2,3,0.592002,0.264261,-0.231615,-0.769368,0.607812,0.004029,0.678765,...,-0.564400,-12.941784,1.521937,4.821578,0.257725,0.104134,0.914120,-11.763607,117.453835,pop
3,/Users/carson/aml-music/Data/music_extracted/p...,3,3,0.658083,0.203790,0.032818,-0.840015,0.642871,0.161227,0.679449,...,-2.277856,-12.339119,1.721230,3.794367,0.041300,0.739907,1.821746,-10.733788,123.046875,pop
4,/Users/carson/aml-music/Data/music_extracted/p...,4,3,0.650039,0.207595,0.011922,-0.806134,0.641716,0.101101,0.688687,...,-3.659871,-14.667474,2.106563,4.672123,0.001524,0.324712,1.533571,-11.142745,123.046875,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16969,/Users/carson/aml-music/Data/music_extracted/j...,96,3,0.526700,0.293894,0.089637,-1.202188,0.508631,0.018586,0.608127,...,5.517359,-14.402899,3.193712,8.200100,-0.539063,-0.212058,4.330453,-18.791008,99.384014,jazz
16970,/Users/carson/aml-music/Data/music_extracted/j...,97,3,0.407806,0.314810,0.576743,-0.922388,0.334936,0.001068,0.614882,...,2.355895,-12.726092,8.788071,10.787657,0.021738,-0.075349,8.748662,-17.626427,99.384014,jazz
16971,/Users/carson/aml-music/Data/music_extracted/j...,98,3,0.460262,0.301435,0.350008,-1.031196,0.416258,0.004966,0.601705,...,0.832232,-21.175072,5.426207,7.388105,-0.470745,-0.153447,6.352828,-16.710958,95.703125,jazz
16972,/Users/carson/aml-music/Data/music_extracted/j...,99,3,0.499169,0.293126,0.176468,-1.109606,0.478375,0.013097,0.609310,...,5.099694,-25.803864,8.008111,9.928221,-0.703184,1.116137,8.035803,-29.178545,99.384014,jazz


In [16]:
df.to_csv('custom_song_features.csv')